In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import random
import string

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
gis_server = config['srv']['server']
pip_server = config['srv']['pip']
gis = config['db']['parksgis']
pip = config['db']['pip']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + pip_server +';Database=' + pip + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
pip_engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [5]:
con_string = 'Driver={' + driver + '};Server=' + gis_server +';Database=' + gis + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
gis_engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Set the prop id lookup field dictionary

In [6]:
#Define the dict of dicts that contains the feature classes and the translated column names
field_lookup = {'property_evw': {'propnum': 'gispropnum',
                                 'prop id': 'gispropnum', 
                                 'borough': 'department',
                                 'ampsdistrict': 'department',
                                 'prop name': 'signname',
                                 'site name': 'signname',
                                 'prop location': 'location',
                                 'site location': 'location',
                                 'acres': 'acres', 
                                 'jurisdiction': 'jurisdiction', 
                                 'typecategory': 'typecategory', 
                                 'featurestatus':'featurestatus', 
                                 'gisobjid':'gisobjid'}, 
                 'playground_evw': {'propnum': 'parentid', 
                                    'prop id': 'omppropid', 
                                    'borough': 'department',
                                    'ampsdistrict': 'department',
                                    'site name': 'signname', 
                                    'site location': 'location', 
                                    'acres': 'acres', 
                                    'jurisdiction': 'jurisdiction', 
                                    'featurestatus':'featurestatus', 
                                    'gisobjid':'gisobjid'},  
                 'zone_evw': {'propnum': 'parentid', 
                              'prop id': 'omppropid', 
                              'borough': 'department',
                              'ampsdistrict': 'department',
                              'site name': 'sitename', 
                              'site location': 'location', 
                              'acres': 'acres', 
                              'jurisdiction': 'jurisdiction', 
                              'featurestatus':'featurestatus', 
                              'gisobjid':'gisobjid'}, 

                 'unmapped_gisallsites_evw': {'propnum': 'gispropnum',
                                              'prop id': 'omppropid', 
                                              'borough': 'department',
                                              'ampsdistrict': 'department',
                                              'site name': 'name',
                                              'site location': 'location',
                                              'acres': 'acres', 
                                              'jurisdiction': 'jurisdiction'},
                 'schoolyard_to_playground_evw': {'propnum': 'gispropnum',
                                                  'prop id': 'gispropnum', 
                                                  'borough': 'department',
                                                  'ampsdistrict': 'department',
                                                  'prop name': 'signname',
                                                  'site name': 'signname',
                                                  'prop location': 'location',
                                                  'site location': 'location',
                                                  'acres': 'acres', 
                                                  'jurisdiction': 'jurisdiction', 
                                                  'featurestatus':'featurestatus'},
                 'greenstreet_evw': {'propnum': 'gispropnum', 
                                     'prop id': 'omppropid', 
                                     'borough': 'department',
                                     'ampsdistrict': 'department',
                                     'site name': 'sitename', 
                                     'site location': 'location', 
                                     'acres': 'acres', 
                                     'jurisdiction': 'jurisdiction', 
                                     'featurestatus':'featurestatus', 
                                     'gisobjid':'gisobjid'},
                 'golfcourse_evw': {'propnum': 'gispropnum', 
                                    'prop id': 'omppropid', 
                                    'borough': 'department',
                                    'ampsdistrict': 'department',
                                    'site name': 'name', 
                                    'site location': 'location', 
                                    'acres': 'acres', 
                                    'jurisdiction': 'jurisdiction', 
                                    'featurestatus':'featurestatus', 
                                    'gisobjid':'gisobjid'},
                 'restrictivedeclarationsite_evw': {'propnum': 'gispropnum',
                                                    'prop id': 'gispropnum', 
                                                    'borough': 'department',
                                                    'ampsdistrict': 'department',
                                                    'prop name': 'signname',
                                                    'site name': 'signname',
                                                    'prop location': 'location',
                                                    'site location': 'location',
                                                    'acres': 'acres', 
                                                    'jurisdiction': 'jurisdiction', 
                                                    'featurestatus':'featurestatus'}, 
                 'structure_evw': {'propnum': 'gispropnum',
                                   'prop id': 'omppropid', 
                                   'borough': 'department',
                                   'ampsdistrict': 'department',
                                   'prop name': 'description',
                                   'site name': 'description',
                                   'prop location': 'location',
                                   'site location': 'location',
                                   'jurisdiction': 'jurisdiction',  
                                   'featurestatus':'featurestatus', 
                                   'gisobjid':'gisobjid'}}

In [9]:
#Define a dict of source feature classes that map to the sourcefc value
sourcefc_lookup = {'property_evw': 'Property', 
                   'playground_evw': 'Playground', 
                   'zone_evw': 'Zone', 
                   'unmapped_gisallsites_evw': 'Unmapped',
                   'schoolyard_to_playground_evw': 'Schoolyard To Playground',
                   'greenstreet_evw': 'Greenstreet',
                   'golfcourse_evw': 'GolfCourse',
                   'restrictivedeclarationsite_evw': 'RestrictiveDeclarationSite',
                   'structure_evw': 'Structure'}

In [10]:
field_lookup_as = {f: ['['+ i[1] + '] as ' + '['+ i[0] + ']' for i in field_lookup[f].items()] for f in sourcefc_lookup }

## Original GIS Data

In [11]:
#Define the tables that will be queried and interacted with
gis_tables = ['property_evw', 'playground_evw', 'zone_evw', 'unmapped_gisallsites_evw', 
              'schoolyard_to_playground_evw', 'greenstreet_evw', 'golfcourse_evw', 
              'restrictivedeclarationsite_evw', 'structure_evw']

In [12]:
#Create the list of SQL Queries
gis_sql_list = ["select objectid, {}, '{}' as sourcefc from parksgis.dpr.{}"
                .format(' ,'.join(field_lookup_as[t]), sourcefc_lookup[t], t) 
                        for t in gis_tables]

In [13]:
#Create a dictionary with sources and dataframes that contain the original data
gis_df_list = {s: pd.read_sql(con = gis_engine, sql = q) for q, s in zip(gis_sql_list, gis_tables)}

## Execute all the stored procedures before starting

In [108]:
db_sql = ['exec accessnewpip.dbo.usp_i_tbl_temp_ref_allsites', 'exec accessnewpip.dbo.usp_m_tbl_ref_allsites',
          'exec accessnewpip.dbo.usp_m_tbl_ref_allsites_nosync', 'exec accessnewpip.dbo.usp_m_tbl_pip_allsites']

In [17]:
#Loop through and execute the update queries
for q in db_sql:
    pip_engine.connect().execute(q)

## Original PIP Data

In [18]:
#Define the tables that will be queried and interacted with
pip_tables = ['tbl_ref_allsites', 'tbl_ref_allsites_nosync', 'tbl_pip_allsites', 
              'vw_pip_compatible_inspected_sites', 'allsites']

In [19]:
#Create the list of SQL Queries
pip_sql_list = ['select * from accessnewpip.dbo.'+ t for t in pip_tables]

In [20]:
#Create a list of dataframes with the original dat
pip_df_list = {s: pd.read_sql(con = pip_engine, sql = q) for q, s in zip(pip_sql_list, pip_tables)}

## Invert the field lookup

In [21]:
inverted_field_lookup = {g:{v:k for k, v in field_lookup[g].items()} for g in gis_tables}

## Copy the DFs

In [23]:
n_rows = {g: random.randint(5, 10) for g in gis_tables}

In [24]:
gis_dfs_update_delete = {g: gis_df_list[g].copy().head(n_rows[g] * 2) for g in gis_tables}

In [25]:
gis_dfs_update = {g: gis_dfs_update_delete[g].copy().head(n_rows[g]) for g in gis_tables}

In [26]:
gis_dfs_delete = {g: gis_dfs_update_delete[g].copy().tail(n_rows[g]) for g in gis_tables}

In [104]:
gis_dfs_insert = {g: gis_df_list[g].copy().tail(n_rows[g]) for g in gis_tables}

## Perform DML Steps

### Delete Records

In [28]:
list_props = {g: list(gis_dfs_delete[g]['prop id']) for g in gis_tables}

In [29]:
where_list = {g: ','.join(f"'{p}'" for p in list(list_props[g])) for g in gis_tables}

In [30]:
#Define the common where clause using the prop id field lookup for each source GIS feature class
where = {f: str(r"where {} in({})").format(field_lookup[f]['prop id'], where_list[f]) for f in field_lookup}

In [31]:
#Loop through the GIS datasets, drop the _evw and create the queries that perform the deletes
sql = ['delete from parksgis.dpr.{} {}'.format(g.replace('_evw', ''), where[g]) for g in gis_tables]

In [32]:
gis_con = gis_engine.connect()

In [33]:
#Loop through and execute the delete queries
for q in sql:
    gis_con.execute(q)

In [34]:
#Create a list of dataframes with the data after deletes, notably all dfs should be empty
gis_df_post_delete = {g: gis_df_list[g][gis_df_list[g]['prop id'].isin(list_props[g])] for g in gis_tables}

### Update Records

In [35]:
list_props = {g: list(gis_dfs_update[g]['prop id']) for g in gis_tables}

In [36]:
where_list = {g: ','.join(f"'{p}'" for p in list(list_props[g])) for g in gis_tables}

In [37]:
#Define the common where clause using the prop id field lookup for each source GIS feature class
where = {f: str(r"where {} in({})").format(field_lookup[f]['prop id'], where_list[f]) for f in field_lookup}

In [38]:
set_value = {g: str(r"[{}] = 'Testing Updates'").format(field_lookup[g]['site location']) for g in gis_tables}

In [39]:
#Loop through the GIS datasets, drop the _evw and perform the deletes
sql = ['update parksgis.dpr.{} set {}  {}'.format(g.replace('_evw', ''), set_value[g], where[g]) for g in gis_tables]

In [40]:
#Loop through and execute the update queries
for q in sql:
    gis_con.execute(q)

In [41]:
#Create a list of dataframes with the data after deletes, notably all dfs should be empty
gis_df_post_update = {g: gis_df_list[g][gis_df_list[g]['prop id'].isin(list_props[g])] for g in gis_tables}

### Insert Non-Duplicate Records

In [43]:
def gen_propid(sourcefc):
    boros = ['B','Q','M','R','X']

    boro = boros[random.randint(0,len(boros) - 1)]
    
    num = f'{random.randint(0, 999):03}'
    
    letters = list(string.ascii_uppercase)
    
    suffixes = ['', letters[random.randint(0,len(letters) - 1)]]
    
    suffix = suffixes[random.randint(0,len(suffixes) - 1)]
    
    suffix_num = f'{random.randint(0, 99):02}'
    
    if sourcefc == 'zone_evw':
        propid = boro + num + suffix + '-ZN' + suffix_num
        
    elif sourcefc == 'playground_evw':
        propid = boro + num + suffix + '-' + suffix_num
        
    elif sourcefc == 'greenstreet_evw':     
        propid = boro + 'Z' + num + suffix
        
    else:
        propid = boro + num + suffix
        
    return propid

In [51]:
def get_value(sourcefc):
    for k, v in field_lookup[sourcefc].items():
        if k == 'prop id':
            return v

In [52]:
apply_cols = {g:[k for k, v in field_lookup[g].items() if v in get_value(g)] for g in gis_tables }

In [70]:
gis_df_list['property_evw']['objectid'].max()

97348

In [105]:
for g in gis_tables:
    
    #Take the existing max objectid and add that value to the current objectid
    objectid = gis_df_list[g].copy()['objectid'].max()
    gis_dfs_insert[g]['objectid'] = gis_dfs_insert[g].apply(lambda x: x['objectid'] + objectid, axis = 1)
    
    #Add a globalid column because it's required
    gis_dfs_insert[g]['globalid'] = gis_dfs_insert[g].apply(lambda x: str(uuid.uuid4()), axis = 1)
    
    #Generate the prop ids for all relevant columns
    for c in apply_cols[g]:
        gis_dfs_insert[g][c] = gis_dfs_insert[g].apply(lambda x: gen_propid(g), axis = 1)
        
    #Rename the columns to the original names
    gis_dfs_insert[g].rename(columns = field_lookup[g], inplace = True)
    
    #Remove duplicate columns and drop the sourcefc column
    gis_dfs_insert[g] = gis_dfs_insert[g].loc[:,~gis_dfs_insert[g].columns.duplicated()].drop(columns = ['sourcefc'])

In [99]:
import uuid

In [107]:
for g in gis_tables:
    fc = g.replace('_evw', '')
    gis_dfs_insert[g].to_sql(fc, gis_con, schema = 'dpr', if_exists = 'append', index = False)

## Execute the PIP Sync

In [109]:
#Loop through and execute the update queries
for q in db_sql:
    pip_engine.connect().execute(q)

## Get the PIP and GIS Tables after

In [110]:
#Create a list of dataframes with the original dat
pip_df_list_after = {s: pd.read_sql(con = pip_engine, sql = q) for q, s in zip(pip_sql_list, pip_tables)}

### Insert Duplicate Records